# Лабораторная работа №2

Выполнили:
* Меньшутин Алексей, M3436
* Каменев Юрий, M3436

In [2]:
import numpy as np

Возьмём три функции:
* квадратичная: $f(\tilde{x}) = 100(x_2 - x_1) ^ 2 + (1 - x_1) ^ 2$
* функция Розенброка: $f(\tilde{x}) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2 $
* тестовая функция: $f(x, y) = 2e^{-(\frac{x - 1}{2})^2 - (\frac{y - 1}{1})^2} + 3e^{-(\frac{x - 2}{3})^2 - (\frac{y - 3}{2})^2} $

In [3]:
def quad_function(x):
    return 100 * (x[1] - x[0]) ** 2 + (1 - x[0]) ** 2

def rosenbrock(x):
    return 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2

def rosenbrock_function_grad(arg):
    x = arg[0]
    y = arg[1]
    dx = 2 * (200 * (x ** 3) - 200 * x * y + x - 1)
    dy = 200 * (y - (x ** 2))
    return np.array([dx, dy])


def test_function(x, y):
    return 2 * np.exp(-((x - 1) / 2) ** 2 - ((y - 1) / 1) ** 2) + 3 * np.exp(-((x - 2) / 3) ** 2 - ((y - 3) / 2) ** 2)


## Метод сопряжённых направлений

Метод Флетчера-Ривса

In [4]:
def golden_ratio(function, left_border, right_border, x_n, s_n, eps=1e-5):
    phi = (1 + np.sqrt(5)) / 2
    a = left_border
    b = right_border

    c_1 = b - (b - a) / phi
    c_2 = a + (b - a) / phi

    calc_result = [function(x_n + c_1 * s_n), function(x_n + c_2 * s_n)]

    while (b - a) / 2 >= eps:
        if calc_result[0] > calc_result[1]:
            a = c_1
            c_1 = c_2
            c_2 = b - (c_1 - a)
            calc_result[0] = calc_result[1]
            calc_result[1] = function(x_n + c_2 * s_n)
        else:
            b = c_2
            c_2 = c_1
            c_1 = a + b - c_2
            calc_result[1] = calc_result[0]
            calc_result[0] = function(x_n + c_1 * s_n)

    c = (b + a) / 2

    return c

def flercherRievesMethod(n, f, f_grad, start_point, eps = 1e-5):
    x_k = start_point

    num_iter = 0

    anti_grad_x_k = -f_grad(x_k)
    alpha_k = golden_ratio(f, 0, 1e5, x_k, anti_grad_x_k, 1e-8)
    s_prev = anti_grad_x_k
    grad_square = np.inner(anti_grad_x_k, anti_grad_x_k)
    x_k = x_k + alpha_k * anti_grad_x_k

    while grad_square > eps:
        num_iter += 1

        prev_grad = anti_grad_x_k
        anti_grad_x_k = -f_grad(x_k)
        if num_iter == n * 5:
            beta_k = 0
            num_iter = 0
        else:
            beta_k = np.inner(anti_grad_x_k, anti_grad_x_k) / np.inner(prev_grad, prev_grad)
        s_k = anti_grad_x_k + beta_k * s_prev
        s_prev = s_k
        alpha_k = golden_ratio(f, 0, 1e5, x_k, s_k)
        x_k = x_k + alpha_k * s_k

    print(num_iter)
    return x_k

flercherRievesMethod(2, rosenbrock, rosenbrock_function_grad, np.zeros(2))

## Метод Ньютона

Описание...